In [2]:
import numpy as np
import pandas as pd
import glob
import MeCab
import re
import nltk
import traceback
#from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus.reader.wordnet import WordNetError
from nltk.corpus import wordnet as wn
from nltk import *


frame = pd.DataFrame()
list_jp = []
list_merge = []
list_en = []
month_start = 1
month_end =  12
year = 2014
period = ["%.2d" % i for i in range(month_start, month_end+1)]

column_need = ["PNAC", "UNIQUE_STORY_INDEX", "HEADLINE_ALERT_TEXT", "TAKE_TEXT",
               "corrected","update_code","reference_code"]
column_need_JP = ["PNAC_JP", "UNIQUE_STORY_INDEX_JP", "HEADLINE_ALERT_TEXT_JP",
                  "TAKE_TEXT_JP","UPDATE_CODE_JP"]
column_need_EN = ["PNAC_EN", "UNIQUE_STORY_INDEX_EN", "HEADLINE_ALERT_TEXT_EN",
                  "TAKE_TEXT_EN","UPDATE_CODE_EN"]

base_path = "/data/data_wzh/RTRS/"+str(year)+"/"
work_path = "~/research/data_prepare/"
file_name = str(year)+"_merge_jp_en.csv"
file_name2 = str(year)+"_cleaned_jp_en.csv"


In [53]:
    for month in period:
        # Load the file monthly
        file_path = base_path + "rna002_RTRS_" + str(year) + "_" + month + ".csv"
        try :
            df = pd.read_csv(file_path, index_col=None, header=0)

            # Filter out the Japanese articles
            df_month_jp = df[df["LANGUAGE"].isin(["JA"])
                             & df["EVENT_TYPE"].isin(["STORY_TAKE_OVERWRITE"])
                             & df["TAKE_TEXT"].str.contains("参照番号")]

            # Filter out the English articles
            df_month_en = df[df["LANGUAGE"].isin(["EN"])
                             & df["EVENT_TYPE"].isin(["STORY_TAKE_OVERWRITE"])]

            # Extract the reference code for Japanese news [会有警告，为何？]
            df_month_jp["reference_code"] = df_month_jp["TAKE_TEXT"].str.extract('参照番号\\[([\\w]+)\\]')

            # Drop the row where reference code is NaN
            df_month_jp = df_month_jp.dropna(subset=["reference_code"])
            
            # Extract update code for all news
            df_month_en["update_code"] = df_month_en["HEADLINE_ALERT_TEXT"].str.extract('UPDATE[\s]*([1-9])')
            df_month_jp["update_code"] = df_month_jp["HEADLINE_ALERT_TEXT"].str.extract('UPDATE[\s]*([1-9])')
            
            df_month_en["update_code"] = df_month_en["update_code"].fillna(value = 1)
            df_month_jp["update_code"] = df_month_jp["update_code"].fillna(value = 1)
            
            #Extract part of "CORRECTED"
            df_month_en["corrected"] = df_month_en["HEADLINE_ALERT_TEXT"].str.extract('(CORRECTED)')
            df_month_jp["corrected"] = df_month_jp["HEADLINE_ALERT_TEXT"].str.extract('(訂正)')
            
            df_month_en["corrected"] = df_month_en["corrected"].where(df_month_en["corrected"].isnull(), 1)
            df_month_jp["corrected"] = df_month_jp["corrected"].where(df_month_jp["corrected"].isnull(), 1)
            
            df_month_en["corrected"] = df_month_en["corrected"].fillna(value = 0)
            df_month_jp["corrected"] = df_month_jp["corrected"].fillna(value = 0)
            
            
            # Find the English news basing on the Japanese news
            df_month_merge = pd.merge(df_month_jp[column_need].reset_index(),
                                      df_month_en[column_need[:-1]].reset_index(),
                                      left_on=['reference_code', "update_code","corrected"],
                                      right_on=['PNAC',"update_code","corrected"],
                                      how="left").dropna()
            list_merge.append(df_month_merge)

            # # Append the Data frame to the list
            # list_jp.append(df_month_jp[column_need])
            # list_en.append(df_month_en[column_need[:-1]])
        except BaseException:
            print(("[I] Error! in extraction for month", month))
            print ('traceback.format_exc():\n%s' % traceback.format_exc())
            print ('########################################################') 
        else:
            print ("[I] Finish referenece extraction for month", month)

    # Merge to one dataframe
    df_merge = pd.concat(list_merge)

    # Save to a csv file
    df_merge.to_csv(work_path + file_name)


/home/R2016hwang/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/home/R2016hwang/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/R2016hwang/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/home/R2016hwang/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_l

[I] Finish referenece extraction for month 01
[I] Finish referenece extraction for month 02
[I] Finish referenece extraction for month 03
[I] Finish referenece extraction for month 04
[I] Finish referenece extraction for month 05
[I] Finish referenece extraction for month 06
[I] Finish referenece extraction for month 07
[I] Finish referenece extraction for month 08
[I] Finish referenece extraction for month 09
[I] Finish referenece extraction for month 10
[I] Finish referenece extraction for month 11
[I] Finish referenece extraction for month 12


In [54]:
df_month_en

,DATE,TIME,UNIQUE_STORY_INDEX,EVENT_TYPE,PNAC,STORY_DATE_TIME,TAKE_DATE_TIME,HEADLINE_ALERT_TEXT,ACCUMULATED_STORY_TEXT,TAKE_TEXT,...,TOPICS,RELATED_RICS,NAMED_ITEMS,HEADLINE_SUBTYPE,STORY_TYPE,TABULAR_FLAG,ATTRIBUTION,LANGUAGE,update_code,corrected
6,2014-12-01,00:00:00.115,20141201000000nS7N0OL013,STORY_TAKE_OVERWRITE,nS7N0OL013,2014-12-01 00:00:00,NaN,S.Korea Nov manufacturing PMI up but contracts...,NaN,"SEOUL, Dec 1 (Reuters) - South Korea's man...",...,MCE ECI EMRG KR ASIA IPR TRACC PMI NEWS1 APOLL...,KRPMI=ECI,NaN,NaN,S,False,RTRS,EN,1,0
12,2014-12-01,00:00:00.271,20141201000000nZYN1ZBR00,STORY_TAKE_OVERWRITE,nZYN1ZBR00,2014-12-01 00:00:00,NaN,"Test, Please Ignore -- EMEA -- 12/01 00:00",NaN,"Test, Please Ignore -- EMEA -- 12/01 00:00",...,AGA LEN RTRS,NaN,NaN,NaN,S,False,RTRS,EN,1,0
20,2014-12-01,00:00:00.353,20141201000000nZZN1ZBR00,STORY_TAKE_OVERWRITE,nZZN1ZBR00,2014-12-01 00:00:00,NaN,"Test, Please Ignore -- APAC -- 12/01 00:00",NaN,"Test, Please Ignore -- APAC -- 12/01 00:00",...,AGA LEN RTRS,NaN,NaN,NaN,S,False,RTRS,EN,1,0
22,2014-12-01,00:00:00.438,20141201000000nZXN1ZBR00,STORY_TAKE_OVERWRITE,nZXN1ZBR00,2014-12-01 00:00:00,NaN,"Test, Please Ignore -- AMERS -- 12/01 00:00",NaN,"Test, Please Ignore -- AMERS -- 12/01 00:00",...,AGA LEN RTRS,NaN,NaN,NaN,S,False,RTRS,EN,1,0
33,2014-12-01,00:00:05.663,20141201000000nS6N0PP00P,STORY_TAKE_OVERWRITE,nS6N0PP00P,2014-12-01 00:00:00,NaN,TABLE-South Korea Nov exports fall 1.9 pct yr/yr,NaN,"SEOUL, Dec 1 (Reuters) - Following are Sou...",...,KR EMRG ASIA MCE NEWS1 TRD ECI LEN RTRS TRACC ...,KRW= 0#KRCOMP1=KQ KREXP=ECI KRIMP=ECI KRTBLP=ECI,NaN,NaN,S,True,RTRS,EN,1,0
35,2014-12-01,00:00:05.676,20141201000005nIGB34403E,STORY_TAKE_OVERWRITE,nIGB34403E,2014-12-01 00:00:05,NaN,CBOT wheat deliveries - Dec 01,NaN,CONTRACT: DECEMBER 2014 KC HRW W...,...,AGRI AMERS COM GRA LEN MEAL OILS US RTRS,NaN,WHE/D,NaN,S,True,RTRS,EN,1,0
37,2014-12-01,00:00:05.690,20141201000005nIFR6LhC17,STORY_TAKE_OVERWRITE,nIFR6LhC17,2014-12-01 00:00:05,NaN,BUZZ-AUD/USD-One way traffic for AUD,NaN,\n\t\t\t\t\t\t\n* AUD back to the top as regio...,...,ASIA AU FRX LEN RTRS,NaN,NaN,NaN,S,True,RTRS,EN,1,0
39,2014-12-01,00:00:07.014,20141201000007nIGB344040,STORY_TAKE_OVERWRITE,nIGB344040,2014-12-01 00:00:07,NaN,CBOT oats deliveries - Dec 01,NaN,CONTRACT: DECEMBER 2014 OATS FUTUR...,...,AGRI AMERS COM GRA LEN MEAL OILS US RTRS,NaN,OATS/DEL,NaN,S,True,RTRS,EN,1,0
41,2014-12-01,00:00:07.604,20141201000007nIGB344041,STORY_TAKE_OVERWRITE,nIGB344041,2014-12-01 00:00:07,NaN,CBOT ethanol deliveries - Dec 01,NaN,CONTRACT: DECEMBER 2014 ETHANOL FU...,...,AGRI AMERS COM GRA LEN MEAL NRG OILS PROD US RTRS,NaN,COR/DEL,NaN,S,True,RTRS,EN,1,0
43,2014-12-01,00:00:08.642,20141201000008nIGB344042,STORY_TAKE_OVERWRITE,nIGB344042,2014-12-01 00:00:08,NaN,CBOT corn deliveries - Dec 01,NaN,CONTRACT: DECEMBER 2014 CORN FUT...,...,AGRI AMERS COM GRA LEN MEAL NRG OILS PROD US RTRS,NaN,COR/DEL,NaN,S,True,RTRS,EN,1,0


In [55]:
print(df_month_en.iloc[0]["HEADLINE_ALERT_TEXT"])

S.Korea Nov manufacturing PMI up but contracts for third month


In [56]:
df_month_jp

,DATE,TIME,UNIQUE_STORY_INDEX,EVENT_TYPE,PNAC,STORY_DATE_TIME,TAKE_DATE_TIME,HEADLINE_ALERT_TEXT,ACCUMULATED_STORY_TEXT,TAKE_TEXT,...,RELATED_RICS,NAMED_ITEMS,HEADLINE_SUBTYPE,STORY_TYPE,TABULAR_FLAG,ATTRIBUTION,LANGUAGE,reference_code,update_code,corrected
91,2014-12-01,00:01:46.506,20141201000146nL3N0TK0VD,STORY_TAKE_OVERWRITE,nL3N0TK0VD,2014-12-01 00:01:46,NaN,BUZZ-〔外為マーケットアイ〕ユーロ／スイスフランテクニカル分析：マドを開けて始まる,NaN,［東京 １日 ロイター］ - \n BUZZ-ユーロ／スイスフランテクニカル...,...,NaN,JPY/JU,NaN,S,False,RTRS,JA,nIFR2kMRrB,1,0
126,2014-12-01,00:05:12.906,20141201000512nL3N0TK0UA,STORY_TAKE_OVERWRITE,nL3N0TK0UA,2014-12-01 00:05:12,NaN,香港デモ隊と警察が衝突、政府本部庁舎近く 少なくとも18人逮捕,NaN,［香港 １日 ロイター］ - 民主派の大規模デモが続く香港で１日未明、政府本部庁舎を...,...,NaN,NaN,NaN,S,False,RTRS,JA,nL3N0TK0AQ,1,0
166,2014-12-01,00:15:11.328,20141201001511nL3N0TK0TK,STORY_TAKE_OVERWRITE,nL3N0TK0TK,2014-12-01 00:15:11,NaN,米感謝祭週末の小売店舗売上高が減少、セール前倒しや通販利用増で,NaN,［３０日 ロイター］ - 全米小売業協会（ＮＲＦ）が３０日発表したデータによると、年...,...,M.N WMT.N BBY.N TGT.N,NaN,NaN,S,False,RTRS,JA,nL2N0TK0MQ,1,0
200,2014-12-01,00:19:03.553,20141201001903nL3N0TL01P,STORY_TAKE_OVERWRITE,nL3N0TL01P,2014-12-01 00:19:03,NaN,メキシコ国営石油、10月の原油生産は過去最低,NaN,［メキシコ市 ２８日 ロイター］ - メキシコの国営石油会社ペメックス[PEMX.U...,...,PEMX.UL,NaN,NaN,S,False,RTRS,JA,nL2N0TJ00Q,1,0
208,2014-12-01,00:20:58.022,20141201002058nL3N0TK0P7,STORY_TAKE_OVERWRITE,nL3N0TK0P7,2014-12-01 00:20:58,NaN,米ＴＰＧ、中国自動車販売会社ＣＧＡの全株式売却へ,NaN,［香港 ２８日 ロイター］ - 米プライベートエクイティ（ＰＥ）のＴＰＧキャピタルは...,...,600837.SS 0665.HK,NaN,NaN,S,False,RTRS,JA,nL6N0TI37D,1,0
401,2014-12-01,00:43:35.653,20141201004335nL3N0TL057,STORY_TAKE_OVERWRITE,nL3N0TL057,2014-12-01 00:43:35,NaN,豪民間インフレ指数、11月は前月比0.1％上昇に鈍化,NaN,［シドニー １日 ロイター］ - ＴＤセキュリティーズとメルボルン研究所が発表した１...,...,AUTDMI=ECI,NaN,NaN,S,False,RTRS,JA,nS9N0QN00G,1,0
422,2014-12-01,00:46:22.425,20141201004622nL3N0TL06P,STORY_TAKE_OVERWRITE,nL3N0TL06P,2014-12-01 00:46:22,NaN,BUZZ-〔外為マーケットアイ〕ドル／オフショア人民元、6.13─6.18元の上限トライへ,NaN,［東京 １日 ロイター］ - \n BUZZ-ドル／オフショア人民元、6.1...,...,NaN,JPY/JU,NaN,S,False,RTRS,JA,nIFR4TPZtx,1,0
429,2014-12-01,00:47:29.567,20141201004729nL3N0TL05I,STORY_TAKE_OVERWRITE,nL3N0TL05I,2014-12-01 00:47:29,NaN,ＥＣＢの追加緩和「ハードル高い」＝ラウテンシュレーガー専務理事,NaN,［ベルリン ２９日 ロイター］ - 欧州中央銀行（ＥＣＢ）のラウテンシュレーガー専務...,...,NaN,NaN,NaN,S,False,RTRS,JA,nL6N0TJ06K,1,0
452,2014-12-01,00:50:11.439,20141201005011nL3N0TK0RO,STORY_TAKE_OVERWRITE,nL3N0TK0RO,2014-12-01 00:50:11,NaN,クラブメッド買収合戦、復星が1日に条件引き上げへ＝関係筋,NaN,［パリ ３０日 ロイター］ - 仏リゾート施設運営会社クラブメッド<CMIP.PA>...,...,CMIP.PA 0656.HK,NaN,NaN,S,False,RTRS,JA,nL6N0TK0VN,1,0
454,2014-12-01,00:50:13.394,20141201005013nL3N0TL078,STORY_TAKE_OVERWRITE,nL3N0TL078,2014-12-01 00:50:13,NaN,韓国サムスン電子、退任観測のモバイル部門トップが留任,NaN,［ソウル １日 ロイター］ - 韓国サムスン電子<005930.KS>のモバイル部門...,...,005930.KS,NaN,NaN,S,False,RTRS,JA,nL3N0TI1VM,1,0


In [57]:
df_month_jp['corrected'].value_counts()

0    3710
1      21
Name: corrected, dtype: int64

In [58]:
df_month_en['corrected'].value_counts()

0    114574
1       408
Name: corrected, dtype: int64

In [59]:
df_month_jp["corrected"].mask(df_month_jp["corrected"]!=0, 1).value_counts()

0    3710
1      21
Name: corrected, dtype: int64

In [60]:
df_month_merge

,index_x,PNAC_x,UNIQUE_STORY_INDEX_x,HEADLINE_ALERT_TEXT_x,TAKE_TEXT_x,corrected,update_code,reference_code,index_y,PNAC_y,UNIQUE_STORY_INDEX_y,HEADLINE_ALERT_TEXT_y,TAKE_TEXT_y
9,454,nL3N0TL078,20141201005013nL3N0TL078,韓国サムスン電子、退任観測のモバイル部門トップが留任,［ソウル １日 ロイター］ - 韓国サムスン電子<005930.KS>のモバイル部門...,0,1,nL3N0TI1VM,188.0,nL3N0TI1VM,20141201000424nL3N0TI1VM,Samsung Elec retains mobile division chief - S...,"SEOUL, Dec 1 (Reuters) - South Korean cong..."
12,531,nL3N0TL07Q,20141201005940nL3N0TL07Q,米原油先物が64.10ドルまで下落、北海ブレントも68ドル台に,［シンガポール １日 ロイター］ - １日のアジア時間の取引で、米原油先物が２ドル超...,0,1,nL3N0TL00O,238.0,nL3N0TL00O,20141201002455nL3N0TL00O,U.S. crude hits five-year low on market rout,* U.S. crude falls near $64/bbl; Brent sli...
14,611,nZUN8GBR00,20141201005904nZUN8GBR00,中国国家統計局の製造業ＰＭＩ、11月は50.3 予想以上の低下,［北京 １日 ロイター］ - 中国国家統計局が発表した１１月の中国製造業購買担当者景...,0,1,nZZN3DBR00,2002.0,nZZN3DBR00,20141201005904nZZN3DBR00,RPT--China's official PMI slips to 50.3 in Nov...,"(Repeats to attach to alert) \n BEIJING, D..."
20,2798,nL3N0TL0YY,20141201011827nL3N0TL0YY,BUZZ-〔外為マーケットアイ〕米ドル／加ドルテクニカル分析：強気シグナル、11月高値意識,［東京 １日 ロイター］ - \n BUZZ-米ドル／加ドルテクニカル分析：...,0,1,nIFR9c3PW6,481.0,nIFR9c3PW6,20141201005412nIFR9c3PW6,BUZZ-USD/CAD-Novenber high in view amid positi...,\n\t\t\t\t\t\t\n* +35pts in Asia with USD & so...
21,2798,nL3N0TL0YY,20141201011827nL3N0TL0YY,BUZZ-〔外為マーケットアイ〕米ドル／加ドルテクニカル分析：強気シグナル、11月高値意識,［東京 １日 ロイター］ - \n BUZZ-米ドル／加ドルテクニカル分析：...,0,1,nIFR9c3PW6,511.0,nIFR9c3PW6,20141201005412nIFR9c3PW6,BUZZ-USD/CAD-November high in view amid positi...,\n\t\t\t\t\t\t\n* +35pts in Asia with USD & so...
24,2882,nL3N0TL16U,20141201012605nL3N0TL16U,BUZZ-〔外為マーケットアイ〕鉄鉱石先物が約2％安で取引開始、豪ドル／米ドルは安値圏,［東京 １日 ロイター］ - \n BUZZ-鉄鉱石先物が約2％安で取引開始...,0,1,nIFRb0Ppsg,2048.0,nIFRb0Ppsg,20141201011403nIFRb0Ppsg,BUZZ-AUD/USD-Iron ore futures open around 2.0%...,\n\t\t\t\t\t\t\n* Dalian iron ore futures open...
27,3121,nZUN8HBR00,20141201014506nZUN8HBR00,ＨＳＢＣの11月中国製造業ＰＭＩは50.0に低下、6カ月ぶり低水準,［北京 １日 ロイター］ - ＨＳＢＣ／マークイットが発表した１１月の中国製造業購買...,0,1,nS7N0OL012,3071.0,nS7N0OL012,20141201014500nS7N0OL012,China Nov HSBC manufacturing acvitivy falls to...,"BEIJING, Dec 1 (Reuters) - Growth in Chine..."
28,3121,nZUN8HBR00,20141201014506nZUN8HBR00,ＨＳＢＣの11月中国製造業ＰＭＩは50.0に低下、6カ月ぶり低水準,［北京 １日 ロイター］ - ＨＳＢＣ／マークイットが発表した１１月の中国製造業購買...,0,1,nS7N0OL012,3455.0,nS7N0OL012,20141201022654nS7N0OL012,REFILE-China Nov HSBC manufacturing activity f...,(Corrects spelling in headline) \n BEIJING...
29,3123,nL3N0TL1AP,20141201015106nL3N0TL1AP,BUZZ-〔外為マーケットアイ〕ドル／アジア通貨、中国統計局ＰＭＩへの反応は限定的,［東京 １日 ロイター］ - \n BUZZ-ドル／アジア通貨、中国統計局Ｐ...,0,1,nIFR8ZH2C0,1756.0,nIFR8ZH2C0,20141201011216nIFR8ZH2C0,BUZZ-Muted response to official China PMI/HSBC...,\n\t\t\t\t\t\t\n* Muted response so far to wor...
30,3123,nL3N0TL1AP,20141201015106nL3N0TL1AP,BUZZ-〔外為マーケットアイ〕ドル／アジア通貨、中国統計局ＰＭＩへの反応は限定的,［東京 １日 ロイター］ - \n BUZZ-ドル／アジア通貨、中国統計局Ｐ...,0,1,nIFR8ZH2C0,2111.0,nIFR8ZH2C0,20141201011216nIFR8ZH2C0,BUZZ-Muted response to official China PMI/HSBC...,\n\t\t\t\t\t\t\n* Muted response so far to wor...


In [61]:
df_month_merge["corrected"].value_counts()

0    3038
1       7
Name: corrected, dtype: int64